In [2]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls
%mkdir data
!ls

gdrive	sample_data
data  gdrive  sample_data


In [6]:
import shutil
#Para copiar un archivo de drive a data
#shutil.copy("direccion inicial","direccion final")
shutil.copy("/content/gdrive/MyDrive/dataset/oneforall_fetal.csv","/content/data/dat_02.csv")

'/content/data/dat_02.csv'

In [7]:
# Con Numpy
data = np.loadtxt("/content/data/dat_02.csv", delimiter=',')
#Cambiamos estos dos valores al valor n de las x
X, y = data[:, 0:21], data[:, 21]
print(X[0])
print(X)
print(y)

[132.   0.  99.   0.  12.   0.   0.  26.   6.   0.   5. 143.  50. 193.
  10.   0. 133. 113. 117.  89.   0.]
[[132.   0.  99. ... 117.  89.   0.]
 [120.   0.   0. ... 121.  73.   1.]
 [132.   6.   0. ... 140.  12.   0.]
 ...
 [140.   1.   0. ... 152.   4.   1.]
 [140.   1.   0. ... 151.   4.   1.]
 [142.   2.   2. ... 145.   1.   0.]]
[1. 2. 1. ... 2. 2. 1.]


In [8]:
# Son las x que tiene el dataset
input_layer_size  = 21

# se DEFINE CUANTOS ESTADOS DE SALIDA EXISTEN
num_labels = 3

#Numero de filas o ejemplos
m = y.size

#Numero de x es n

In [9]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [10]:
def lrCostFunction(theta, X, y, lambda_):
    
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [11]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [12]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [13]:
def oneVsAll(X, y, num_labels, lambda_):
  
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    #for c in np.arange(num_labels): solo cuando y esta al incio
  # c+1 se hace para sabr los n resltados

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == (c+1)), lambda_), 
                                jac=True, 
                                method='BFGS',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [14]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta)

[[ 6.11673795e+00  6.05559707e-02  9.18559741e-01 -2.86954053e-03
   2.26507700e-01  3.43974381e-02  1.49282126e-01 -3.14885672e+00
  -7.10503032e-02  6.81860178e-01 -3.24336092e-02 -3.45926660e-02
  -2.92758972e-03 -1.76675313e-02 -2.05951210e-02 -8.61925344e-02
  -2.45106493e-02  1.18649211e-01 -3.64829867e-02 -1.14090938e-01
  -2.92973784e-02 -1.65624411e-01]
 [-4.17746808e+00 -9.16007363e-02 -7.73832799e-01 -5.78833868e-03
  -1.03778778e-01  7.74385009e-02 -4.24448434e-02  5.63133347e-01
   1.72401755e-02 -2.28598690e+00  5.86752147e-03  4.85102987e-02
  -6.07310454e-04  1.05451552e-03  4.47205069e-04  2.28119524e-01
   1.06575118e-01  3.22814310e-03  1.06319239e-01  2.34670717e-03
   1.95506792e-02 -4.45777883e-01]
 [-3.00629118e-01  2.01210569e-01 -6.77252373e-01  1.38248424e-02
  -2.31637481e-01 -4.96991255e-02  7.20203045e-02  1.42979588e+00
   1.01637132e-01 -1.03936351e+00  5.78436605e-02  7.99814912e-02
   1.43749097e-02  2.22664301e-02  3.66413398e-02 -5.60373214e-01
   5.9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


In [15]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p + 1

In [19]:
print("ONE VS ALL DE ESTADO DE SALUD FETAL QUE VARIA ENTRE 1 Y 3\n1 = Muy Saludable \n2 = Normal \n3 = Sospechoso (Revision)\n")
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[2121:2127, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
#yPrueba = [2]
#XPrueba = [[123, 0, 0, 0, 0, 0, 0, 70, 0.5, 33, 8.3, 71, 89, 160, 5, 0, 123, 124, 126, 1, 0]]
print(XPrueba)

XPrueba = np.concatenate((np.ones((6, 1)), XPrueba), axis=1)
print(XPrueba.shape)
print(XPrueba)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

#displayData(X[500:510, :])

ONE VS ALL DE ESTADO DE SALUD FETAL QUE VARIA ENTRE 1 Y 3
1 = Muy Saludable 
2 = Normal 
3 = Sospechoso (Revision)

(2127, 21)
Precision del conjuto de entrenamiento: 89.56%
(6, 21)
[[140.    0.    0.    5.    1.    0.    0.   77.    0.7  17.    6.   31.
  124.  155.    2.    0.  145.  143.  145.    2.    0. ]
 [140.    0.    0.    7.    0.    0.    0.   79.    0.2  25.    7.2  40.
  137.  177.    4.    0.  153.  150.  152.    2.    0. ]
 [140.    1.    0.    7.    0.    0.    0.   78.    0.4  22.    7.1  66.
  103.  169.    6.    0.  152.  148.  151.    3.    1. ]
 [140.    1.    0.    7.    0.    0.    0.   79.    0.4  20.    6.1  67.
  103.  170.    5.    0.  153.  148.  152.    4.    1. ]
 [140.    1.    0.    6.    0.    0.    0.   78.    0.4  27.    7.   66.
  103.  169.    6.    0.  152.  147.  151.    4.    1. ]
 [142.    2.    2.    8.    0.    0.    0.   74.    0.4  36.    5.   42.
  117.  159.    2.    1.  145.  143.  145.    1.    0. ]]
(6, 22)
[[  1.  140.    0.    0.    5